### 반도체 박막 두께 알고리즘 분석 대회 By Dacon

In [9]:
import pandas as pd
import warnings
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

In [10]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [11]:
#독립변수와 종속변수를 분리합니다.
train_X = train.iloc[:,4:]  # 독립변수 -> 반사율(0~225)
train_Y = train.iloc[:,0:4] # 종족변수 -> layer 1~4 두께
test_X = test.iloc[:,1:]

In [12]:
##### 케라스를 통해 모델 생성을 시작합니다.

from tensorflow import keras
from keras.layers import BatchNormalization
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation
import matplotlib.pyplot as plt
import numpy as np

In [13]:
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping

# 콜백 정의
callback_list =[
    ReduceLROnPlateau(
        monitor='val_loss',  # 검증 손실을 기준으로 callback이 호출
        factor=0.2,          # callback 호출시 학습률을 1/2로 줄인다.
        patience=5,         # epoch 5 동안 개선되지 않으면 실행
    ),
    ModelCheckpoint(
        filepath='my_model.{epoch:02d}-{val_loss:.4f}.hdf5',  #저장
        monitor='val_loss',
        save_best_only=True,    #가장 좋은 모델
    ),
    EarlyStopping(
        monitor='val_loss',  #epochs 10번동안 val_loss값이 개선되지 않으면 학습종료(과대적합 방지)
        patience=10,
    )
]


In [6]:
# mlp 모델 생성
# 은닉층의 depth와 유닛 node수를 적절히 조정하여 최적의 모델을 찾아야함.
# 학습 효율을 높이기 위해 배치 정규화 - dropout 대체
# 오차 역전파 과정에서 미분한 gradient가 지나치게 커지거나 소실되는 문제를 해결하기위해 가중치 초기화 함수 사용 - he_normal
# 각 층의 활성화 함수의 출력값 분포가 골고루 분포되도록 batch normalization 사용
# ELU는 ReLU의 특성을 공유하고, dead ReLU의 문제점을 해결할 수 있는 활성함수이다.
# 최적화 함수인 Optimaizer는 가장 많이 사용되고있는 adam을 사용하였다.

model = Sequential()

model.add(Dense(512, input_dim = 226, kernel_initializer='he_normal'))
model.add(BatchNormalization())
# 활성함수 
model.add(Activation('elu'))

model.add(Dense(1024, kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('elu'))

model.add(Dense(512,kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('elu'))

model.add(Dense(512,kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('elu'))

model.add(Dense(256,kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('elu'))

model.add(Dense(128,kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('elu'))

model.add(Dense(units=4,kernel_initializer='he_normal', activation='linear'))
          
model.compile(loss='mae', optimizer='adam', metrics=['mae'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               116224    
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              525312    
_________________________________________________________________
batch_normalization_2 (Batch (None, 1024)              4096      
_________________________________________________________________
activation_2 (Activation)    (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)              

In [1]:
# 모델 학습

base_model = model.fit(train_X, train_Y, epochs=150, batch_size=256, validation_split = 0.05, callbacks=callback_list)

NameError: name 'model' is not defined

In [14]:
#예측값을 생성합니다.
pred_test = model.predict(test_X)

In [15]:
#submission 파일을 생성합니다.
sample_sub = pd.read_csv('sample_submission.csv', index_col=0)
submission = sample_sub+pred_test
submission.to_csv('submission.csv')

In [ ]:
# 학습한 모델을 저장. 필요에따라 load하여 학습을 이어갈 수 있음
from keras.models import load_model
model.save('semicon_model.h5')